In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s6e1/sample_submission.csv
/kaggle/input/playground-series-s6e1/train.csv
/kaggle/input/playground-series-s6e1/test.csv


In [2]:
train_df = pd.read_csv('/kaggle/input/playground-series-s6e1/train.csv')
target = 'exam_score'

y = train_df[target]
X = train_df.copy()

X.drop(target,axis=1,inplace=True)

X.head()

,id,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,facility_rating,exam_difficulty
0,0,21,female,b.sc,7.91,98.8,no,4.9,average,online videos,low,easy
1,1,18,other,diploma,4.95,94.8,yes,4.7,poor,self-study,medium,moderate
2,2,20,female,b.sc,4.68,92.6,yes,5.8,poor,coaching,high,moderate
3,3,19,male,b.sc,2.00,49.5,yes,8.3,average,group study,high,moderate
4,4,23,male,bca,7.65,86.9,yes,9.6,good,self-study,high,easy


In [3]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.2, random_state=1,stratify=y)

In [4]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

def OneHot(df,ohe):
    # df = og_df.copy()
    for col in ohe:
        dummy1 = pd.get_dummies(df[col],dtype=int,drop_first=True)
        df = pd.concat([df,dummy1],axis = 1)
        df.drop(col,axis=1,inplace=True)
    return df

cols = ['gender','course','internet_access','sleep_quality','study_method','facility_rating','exam_difficulty']

X_train = OneHot(X_train,cols)
X_val = OneHot(X_val,cols)

ss = ['age','study_hours','class_attendance','sleep_hours']

scaler = StandardScaler()

X_train[ss] = scaler.fit_transform(X_train[ss])
X_val[ss] = scaler.fit_transform(X_val[ss])

X_train.drop('id',axis = 1, inplace=True)
X_val.drop('id',axis = 1, inplace=True)

X_val.head()


,age,study_hours,class_attendance,sleep_hours,male,other,b.sc,b.tech,ba,bba,...,good,poor,group study,mixed,online videos,self-study,low,medium,hard,moderate
271361,1.082877,1.050546,0.459661,0.817045,0,0,0,1,0,0,...,1,0,0,1,0,0,0,0,0,0
370502,1.082877,0.558890,1.273778,-1.419592,0,1,0,0,1,0,...,0,1,1,0,0,0,0,0,0,0
519791,1.525420,-0.212501,1.348310,0.243548,0,1,0,1,0,0,...,0,1,1,0,0,0,1,0,0,1
372006,1.525420,1.652400,0.213132,-0.960795,0,1,0,0,0,0,...,1,0,1,0,0,0,0,0,1,0
251564,-0.687293,0.800480,1.571906,1.046443,0,0,0,0,0,0,...,1,0,0,1,0,0,0,1,0,1


In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

LinReg = LinearRegression()

LinReg.fit(X_train,y_train)
y_hat_train = LinReg.predict(X_train)
y_hat_val = LinReg.predict(X_val)

print("     TRAINING SET")
print(f"R² Score: {r2_score(y_train, y_hat_train):.4f}")
print(f"Mean Squared Error: {mean_squared_error(y_train, y_hat_train):.4f}")
print(f"Mean Absolute Error: {mean_absolute_error(y_train, y_hat_train):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_train, y_hat_train)):.4f}")

print("\n     VALIDATION SET")
print(f"R² Score: {r2_score(y_val, y_hat_val):.4f}")
print(f"Mean Squared Error: {mean_squared_error(y_val, y_hat_val):.4f}")
print(f"Mean Absolute Error: {mean_absolute_error(y_val, y_hat_val):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_val, y_hat_val)):.4f}")

     TRAINING SET
R² Score: 0.7786
Mean Squared Error: 79.2184
Mean Absolute Error: 7.1056
RMSE: 8.9005

     VALIDATION SET
R² Score: 0.7801
Mean Squared Error: 78.6844
Mean Absolute Error: 7.0823
RMSE: 8.8704


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
# from sklearn.model_selection import GridSearchCV

gb_model = GradientBoostingRegressor(
    # verbose=1,
    warm_start=True,
    n_estimators=1000,
    max_depth=4,
    learning_rate=0.1,
    random_state=1
)



gb_model.fit(X_train,y_train)


y_hat_train = gb_model.predict(X_train)
y_hat_val = gb_model.predict(X_val)

print("     TRAINING SET")
print(f"R² Score: {r2_score(y_train, y_hat_train):.4f}")
print(f"Mean Squared Error: {mean_squared_error(y_train, y_hat_train):.4f}")
print(f"Mean Absolute Error: {mean_absolute_error(y_train, y_hat_train):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_train, y_hat_train)):.4f}")

print("\n     VALIDATION SET")
print(f"R² Score: {r2_score(y_val, y_hat_val):.4f}")
print(f"Mean Squared Error: {mean_squared_error(y_val, y_hat_val):.4f}")
print(f"Mean Absolute Error: {mean_absolute_error(y_val, y_hat_val):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_val, y_hat_val)):.4f}")

In [33]:
import xgboost as xgb

xg = xgb.XGBRegressor(
    learning_rate=0.1,
    max_depth=5,
    min_child_weight=7,
    n_estimators=1000,
    random_state=1,
    objective='reg:squarederror',
    eval_metric='rmse'
)

# cv_params = {
#     'max_depth': [5, 8, 16], 
#     'min_child_weight': [5, 7, 9], 
#     'n_estimators': [250, 500, 1000]
# } 

# xgb_cv = GridSearchCV(
#     estimator=xg, 
#     param_grid=cv_params, 
#     scoring='neg_root_mean_squared_error',
#     cv=5,
#     refit=True     
# )

# Optimal parameters: {'max_depth': 5, 'min_child_weight': 7, 'n_estimators': 1000}

xg.fit(X_train,y_train)
# print(f"Optimal parameters: {xgb_cv.best_params_}")

y_hat_train = xg.predict(X_train)
y_hat_val = xg.predict(X_val)

print("     TRAINING SET")
print(f"R² Score: {r2_score(y_train, y_hat_train):.4f}")
print(f"Mean Squared Error: {mean_squared_error(y_train, y_hat_train):.4f}")
print(f"Mean Absolute Error: {mean_absolute_error(y_train, y_hat_train):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_train, y_hat_train)):.4f}")

print("\n     VALIDATION SET")
print(f"R² Score: {r2_score(y_val, y_hat_val):.4f}")
print(f"Mean Squared Error: {mean_squared_error(y_val, y_hat_val):.4f}")
print(f"Mean Absolute Error: {mean_absolute_error(y_val, y_hat_val):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_val, y_hat_val)):.4f}")

     TRAINING SET
R² Score: 0.7946
Mean Squared Error: 73.4890
Mean Absolute Error: 6.8353
RMSE: 8.5726

     VALIDATION SET
R² Score: 0.7859
Mean Squared Error: 76.6138
Mean Absolute Error: 6.9780
RMSE: 8.7529


In [ ]:


test_df = pd.read_csv('/kaggle/input/playground-series-s6e1/test.csv')
test_df = OneHot(test_df, cols)
test_df[ss] = scaler.fit_transform(test_df[ss])

ids = test_df['id']
test_df.drop('id',axis = 1, inplace=True)

y_hat_test = gb_model.predict(test_df)

submission = pd.DataFrame({
    "id": ids,
    "exam_score": y_hat_test
})

submission.to_csv("submission.csv", index=False)